In [1]:
print("Hello")

Hello


In [2]:
%pwd

'/Users/sohamkhopkar/Desktop/Mental Health chatbot/MentalHealthChatbot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/sohamkhopkar/Desktop/Mental Health chatbot/MentalHealthChatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:

extracted_data=load_pdf_file(data='Data/')

In [19]:
# extracted_data

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 3174


In [25]:
# text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings


In [11]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()


/var/folders/vr/k31rtn_168v7kvg7k8xq_v540000gn/T/ipykernel_5234/4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/opt/anaconda3/envs/mindly/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [69]:
# query_result

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.environ.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mindly"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [226]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


In [17]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [20]:
retrieved_docs = retriever.invoke("I am feeling mentally tired")



In [21]:
retrieved_docs

[Document(id='ae1b2e38-84ae-4bf3-b767-ae881b5fe133', metadata={'author': 'Tj Power', 'creationdate': '2025-01-17T15:22:29+00:00', 'creator': 'calibre 7.4.0', 'moddate': '2025-01-17T15:22:29+00:00', 'page': 200.0, 'page_label': '201', 'producer': 'calibre 7.4.0', 'source': 'Data/_OceanofPDF.com_The_DOSE_Effect_-_Tj_Power.pdf', 'title': 'The Dose Effect', 'total_pages': 361.0}, page_content='concentrate.17 Our digitally based lives are incredibly demanding onour attention, constantly having to engage a lot of cognitive e\x00ort.Mental fatigue is something that so many of us experience. Therestorative bene\x00ts of spending time in nature for your brain andbody cannot be overestimated.\nWith a clear understanding that spending time in nature would betransformative for my life, I then had the challenging task of \x00guring'),
 Document(id='23264182-8de8-40c0-85c2-e54870edd8d3', metadata={'author': 'Tj Power', 'creationdate': '2025-01-17T15:22:29+00:00', 'creator': 'calibre 7.4.0', 'moddate

In [23]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="mistral:7b")

In [47]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a supportive and empathetic mental health assistant. "
    "Only answer the question if the provided context includes clear and relevant information. "
    "If the context does not contain enough information, say: \"I'm not sure about that.\" "
    "Never mention the context or documents in your answer. "
    "Respond in a warm, human tone, and keep your answer under three sentences."
    "\n\n"
    "Context:\n{context}"
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [25]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
def answer_user_query(user_input, retriever, model, prompt):
   
    retrieved_docs = retriever.invoke(user_input)

   
    if not retrieved_docs or len(retrieved_docs) == 0:
        return "I'm not sure about that. The provided context doesn't include this information."

    
    question_answer_chain = create_stuff_documents_chain(model, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": user_input})
    return response["answer"]


In [ ]:
response = answer_user_query("", retriever, model, prompt)
print(response)


 Supervised learning refers to a type of machine learning where the model is trained using labeled data, meaning that the correct output (label) for each input is provided during training. In other words, the goal is for the model to learn a mapping from inputs to outputs based on examples it receives. This can be thought of as a teacher providing instruction and feedback to a student about the correct answers to problems they are solving.


In [54]:
user_input = "I am feeling low today?"

In [ ]:
from sentence_transformers import SentenceTransformer, util


embedder = SentenceTransformer("all-MiniLM-L6-v2")


query_embedding = embedder.encode(user_input, convert_to_tensor=True)
retrieved_docs = retriever.invoke(user_input)


relevant_docs = []
for doc in retrieved_docs:
    doc_embedding = embedder.encode(doc.page_content, convert_to_tensor=True)
    similarity = util.cos_sim(query_embedding, doc_embedding).item()
    
    if similarity > 0.5:  
        relevant_docs.append(doc)

if not relevant_docs:
    print("I'm not sure about that. The provided context doesn't include this information.")
else:
    question_answer_chain = create_stuff_documents_chain(model, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    response = rag_chain.invoke({"input": user_input})
    print(response["answer"])


I'm not sure about that. The provided context doesn't include this information.


In [26]:
response = rag_chain.invoke({"input": "What is Supervised Learning"})
print(response["answer"])

 The process of establishing learning goals and objectives for trainees involves defining what information they are expected to acquire. This helps ensure that instruction aligns with the intended learning outcomes. In supervised learning, this might involve understanding how to classify or predict data using a given dataset and specific model architecture.
